In [1]:
import xlsxwriter
from vk import *
import stuffers

T = time.localtime() # struct_time
#T = time.strptime("2017-01-25", "%Y-%m-%d") # для проверки
Tm = T.tm_year*12 + T.tm_mon

if T.tm_mday < 26:
    Tstart = "{0}-{1:0>2}-25 08:00".format( (Tm-3)//12, (Tm-3)%12 + 1)
    Tstop  = "{0}-{1:0>2}-25 08:00".format( (Tm-2)//12, (Tm-2)%12 + 1)
else:
    Tstart = "{0}-{1:0>2}-25 08:00".format( (Tm-2)//12, (Tm-2)%12 + 1)
    Tstop  = "{0}-{1:0>2}-25 08:00".format( (Tm-1)//12, (Tm-1)%12 + 1)
del T, Tm

#Tstart = "2017-02-20 08:00" # тут можно указать временной интервал вручную
#Tstop  = "2017-02-25 08:00" # -//- 

print( "Period: {} - {}".format(Tstart, Tstop) )

FileName = "VK_TagsMonthly_{}_{}.xlsx".format(Tstart[:10], Tstop[:10])
print("File:  ", FileName)

Topics = [#Game
          "https://vk.com/topic-100045377_36678235?offset=5000",
          "https://vk.com/topic-9159189_35234829?offset=0",
          "https://vk.com/topic-29468395_34964992?offset=0",

          #Tech
          "https://vk.com/topic-100045377_36284639?offset=15100",
          "https://vk.com/topic-100045377_36847811?offset=0",
          "https://vk.com/topic-9159189_35234775?offset=0",
          "https://vk.com/topic-29468395_34964999?offset=0",

          #Bill
          "https://vk.com/topic-100045377_36036461?offset=18500",
          "https://vk.com/topic-100045377_36848030?offset=0",
          "https://vk.com/topic-9159189_35234849?offset=0",
          "https://vk.com/topic-29468395_34964977?offset=0",

          #Rule
          "https://vk.com/topic-100045377_35120240?offset=10200",
          "https://vk.com/topic-100045377_36847847?offset=0",
          "https://vk.com/topic-9159189_35234761?offset=0",
          "https://vk.com/topic-29468395_34964997?offset=0"]

Period: 2017-01-25 08:00 - 2017-02-25 08:00
File:   VK_TagsMonthly_2017-01-25_2017-02-25.xlsx


In [2]:
print("Создаём отчёт", FileName)
workbook = xlsxwriter.Workbook( FileName )
worksheet = dict()

print("\nСтраница Common")
worksheet["Common"] = workbook.add_worksheet("Common")
row = 0
for col in range(1,6):
    worksheet["Common"].write(row, col, [u"Users", u"Questions", u"Replies", u"Posts", u"URL"][col-1])
row += 1

print("Выгружаем сообщения из списка тем:")
All_Posts = list()
for topic in Topics:
    
    topic_name = get_group_name(topic) + ' / ' + get_topic_title(topic)
    worksheet["Common"].write(row, 0,  topic_name)
    print(" -", topic_name)
    
    Posts = get_topic_data(topic, stime=Tstart, etime=Tstop)
    
    questions = [p['from_id'] for p in Posts if p['from_id'] not in ADMINS]
    users = frozenset(questions)
    replies = [p['from_id'] for p in Posts if p['from_id'] in ADMINS]
    
    worksheet["Common"].write(row, 1, len(users)) # Users count
    worksheet["Common"].write(row, 2, len(questions)) # Questions count
    worksheet["Common"].write(row, 3, len(replies)) # Replies count
    worksheet["Common"].write(row, 4, len(Posts)) # Posts count
    worksheet["Common"].write(row, 5, find_start_board_offset(topic, start=Tstart) ) # Topic URL
    row += 1
    All_Posts += Posts
worksheet["Common"].write(row, 0, u"Total")
for col in range(1,5):
    worksheet["Common"].write(row, col, u"=SUM({0}{1}:{0}{2})".format(chr(65+col), 2, len(Topics)+1))
print("Страница Common готова!\n")



Создаём отчёт VK_TagsMonthly_2017-01-25_2017-02-25.xlsx

Страница Common
Выгружаем сообщения из списка тем:
 - Центр Поддержки Wargaming.net / Вопросы по игровой механике
 - World of Tanks / ЦПП. Вопросы по игровой механике
 - World of Warships / ЦПП. Вопросы по игровой механике.
 - Центр Поддержки Wargaming.net / Технические вопросы
 - Центр Поддержки Wargaming.net / Технические вопросы
 - World of Tanks / ЦПП. Технические вопросы.
 - World of Warships / ЦПП. Технические вопросы.
 - Центр Поддержки Wargaming.net / Финансовые вопросы/Вопросы по учётной записи
 - Центр Поддержки Wargaming.net / Финансовые вопросы/Вопросы по учётной записи
 - World of Tanks / ЦПП. Финансовые вопросы/Вопросы по учётной записи.
 - World of Warships / ЦПП. Финансовые вопросы/Вопросы по учётной записи.
 - Центр Поддержки Wargaming.net / Вопросы по Правилам Игры
 - Центр Поддержки Wargaming.net / Вопросы по Правилам Игры
 - World of Tanks / ЦПП. Вопросы по Правилам Игры.
 - World of Warships / ЦПП. Вопросы по

In [21]:
#workbook = xlsxwriter.Workbook( FileName )
#worksheet = dict()

In [3]:
print("Содержимое тем загружено. \nПодсчитываем теги... ", end='')
Tags = CountTags(All_Posts)
print("Готово!")

tempList = []
for k in Tags.keys():
    tempList.append((int(re.findall('\d+', k)[0]), k))
stuffList = [k[1] for k in sorted(tempList)]
del tempList

for dep in ["GS", "RS", "BS", "TS"]:
    print("\nФормируем страницу", dep)
    len_dep = 0
    row = 0
    col = 0
    worksheet[dep] = workbook.add_worksheet(dep)
    Columns = ["WoT", "WoWP", "WoWS", "Blitz", "G&G", "Generals", "Other"]

    #Table "Count"
    print("Таблица 'Count'")
    for cell in ["Count"] + Columns + ["All"]:
        worksheet[dep].write(row, col, cell)
        col += 1
    row += 1; col = 0

    order = [str(i) for i in list(range(1, 7)) + [0]]
    for stuff in stuffList:
        if dep in stuff:
            len_dep += 1
            worksheet[dep].write(row, col, stuff); col += 1
            for item in order:
                if item in Tags[stuff]:
                    worksheet[dep].write(row, col, Tags[stuff][item])
                else:
                    worksheet[dep].write(row, col, 0)
                col += 1
            worksheet[dep].write(row, col, "=SUM({1}{0}:{2}{0})".format(row+1, chr(65+1), chr(65+col-1)))
            row += 1; col = 0

    worksheet[dep].write(row, col, "Total"); col += 1
    for col in range(1,len(order) + 2):
        worksheet[dep].write(row, col, "=SUM({0}{1}:{0}{2})".format(chr(65+col), 2, 1+len_dep)) #=SUM(B2:Bxx)

    # Table "%"
    print("Таблица '%'")
    row += 3; col = 0
    for cell in ["%"] + Columns:
        worksheet[dep].write(row, col, cell)
        col += 1
    row += 1; col = 0
    percent_format = workbook.add_format()
    percent_format.set_num_format('0.00%')
    for stuff in stuffList:
        if dep in stuff:
            worksheet[dep].write(row, col, stuff); col += 1
            for col in range(1, len(order)+1):
                worksheet[dep].write(row, col,
                                     "=({1}{0}/{2}{0})".format(row-len_dep-3, chr(65+col), chr(65+len(order)+1)),
                                     percent_format)
            row += 1; col = 0

    worksheet[dep].write(row, col, "Avg"); col += 1
    for col in range(1, len(order) + 1):
        worksheet[dep].write(row, col,
                             "=({1}{0}/{2}{0})".format(row - len_dep - 3, chr(65 + col), chr(65 + len(order) + 1)),
                             percent_format)

    #Table Hours
    print("Таблица 'Hours'")
    row += 3; col = 0
    for cell in ["Time"] + Columns + ["Hours"]:
        worksheet[dep].write(row, col, cell)
        col += 1
    row += 1; col = 0
    hours_format = workbook.add_format()
    hours_format.set_num_format('0.00')
    for stuff in stuffList:
        if dep in stuff:
            #worksheet[dep].write(row, col, stuff)
            try:
                worksheet[dep].write(row, col, stuffers.stuffers[stuff], hours_format)
                worksheet[dep].write(row, len(order)+1, 1, hours_format) # заменить 1 на количество часов
            except:
                worksheet[dep].write(row, col, stuff, hours_format)
                worksheet[dep].write(row, len(order)+1, 1, hours_format) 
            col += 1
            for col in range(1, len(order)+1):
                worksheet[dep].write(row, col, 
                                     "=({0}{1}*{2}{3})".format(chr(65+col), row - len_dep - 3, chr(65+len(order)+1), row+1), 
                                     hours_format )

            row += 1; col = 0

    worksheet[dep].write(row, col, "Total");
    col += 1
    for col in range(1, len(order) + 2):
        worksheet[dep].write(row, col, 
                             "=SUM({0}{1}:{0}{2})".format(chr(65+col), row-len_dep+1, row), #"=({1}{0}/{2}{0})".format(row - len_dep - 3, chr(65 + col), chr(65 + len(order) + 1))
                             hours_format)
                             


workbook.close()
print("\nГотово! Результаты записаны в файл.")

Содержимое тем загружено. 
Подсчитываем теги... Готово!

Формируем страницу GS
Таблица 'Count'
Таблица '%'
Таблица 'Hours'

Формируем страницу RS
Таблица 'Count'
Таблица '%'
Таблица 'Hours'

Формируем страницу BS
Таблица 'Count'
Таблица '%'
Таблица 'Hours'

Формируем страницу TS
Таблица 'Count'
Таблица '%'
Таблица 'Hours'

Готово! Результаты записаны в файл.
